# 0
### A)

In [578]:
from pathlib import Path
from dotenv import load_dotenv
import requests
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [579]:
env_path = Path("../../code_alongs/11_data_from_API/.env")
load_dotenv(env_path)


API_KEY = os.getenv("API_KEY")

url = f"https://api.resrobot.se/v2.1/trip?format=json&originId=740000002&destId=740000003&passlist=true&showPassingPoints=true&accessId={API_KEY}"

response = requests.get(url)

result = response.json()



result.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [580]:
def get_departures(api, id = "740000002"):
    url = f"https://api.resrobot.se/v2.1/departureBoard?id={id}&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()


def get_arrivals(api, id = "740000002"):
    url = f"https://api.resrobot.se/v2.1/arrivalBoard?id={id}&format=json&accessId={api}"
    response = requests.get(url)
    return response.json()

In [581]:
departures = get_departures(api=API_KEY)
arrivals = get_arrivals(api=API_KEY)

In [582]:
departures.keys(), arrivals.keys()

(dict_keys(['Departure', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']),
 dict_keys(['Arrival', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId']))

### B)

In [583]:
len(arrivals["Arrival"])

153

### C)

In [584]:
len(departures["Departure"])

154

### D)

In [585]:
departures["Departure"][0].keys()

dict_keys(['JourneyDetailRef', 'JourneyStatus', 'ProductAtStop', 'Product', 'Notes', 'name', 'type', 'stop', 'stopid', 'stopExtId', 'lon', 'lat', 'time', 'date', 'reachable', 'direction', 'directionFlag'])

In [586]:
for depart in departures["Departure"]:
    if "Spårväg" in depart["name"]:
        print(f"{depart['name']}, {depart['direction']}, {depart['time']}")

Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 12:16:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 12:19:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 12:26:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 12:29:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 12:36:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 12:39:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 12:46:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 12:49:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 12:56:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 12:59:00
Länstrafik - Spårväg 6, Kortedala Aprilgatan (Göteborg kn), 13:06:00
Länstrafik - Spårväg 6, Göteborg Varmfrontsgatan, 13:09:00


### E)

In [587]:
departures = departures["Departure"]

In [588]:
trams = [
    {
        "name": tram.get("name"),
        "lat": tram.get("lat"),
        "lon": tram.get("lon"),
        "direction": tram.get("direction"),
    }
    for tram in departures
    if "Spårväg" in tram.get("name")
]

In [589]:
data= pd.DataFrame(trams)

In [590]:
fig = px.scatter_mapbox(data, lat="lat", lon= "lon", hover_name="direction", mapbox_style= "open-street-map", zoom= 15)

fig.show()

### F)

# 1
### A)

# Stop Lookup

In [591]:
def get_lookup(search):
    url= f"https://api.resrobot.se/v2.1/location.name?input={search}&format=json&accessId={API_KEY}"
    return requests.get(url).json()

In [592]:
lund= get_lookup("Lund")
lund["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098038'

In [593]:
gothenburg= get_lookup("Göteborg")
gothenburg["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098001'

In [594]:
def route_planner(org, des):
    
    url= f"https://api.resrobot.se/v2.1/trip?format=json&originId={org}&destId={des}&passlist=true&showPassingPoints=true&accessId={API_KEY}"

    return requests.get(url).json()

In [595]:
trip= route_planner(740098001, 740098038)

In [596]:
trip.keys()

dict_keys(['Trip', 'ResultStatus', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'planRtTs', 'requestId', 'scrB', 'scrF'])

In [597]:
example_trip = trip["Trip"][0]

In [598]:
stops=example_trip["LegList"]["Leg"][0]["Stops"]["Stop"]
stops

[{'name': 'Göteborg Centralstation',
  'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
  'extId': '740000002',
  'routeIdx': 0,
  'lon': 11.973479,
  'lat': 57.708895,
  'depPrognosisType': 'PROGNOSED',
  'depTime': '12:40:00',
  'depDate': '2025-01-16',
  'depDir': 'Köpenhamn Österport (Danmark)',
  'minimumChangeDuration': 'PT15M'},
 {'name': 'Mölndal station',
  'id': 'A=1@O=Mölndal station@X=12018722@Y=57655958@U=1@L=740000315@',
  'extId': '740000315',
  'routeIdx': 1,
  'lon': 12.018722,
  'lat': 57.655958,
  'depTime': '12:48:00',
  'depDate': '2025-01-16',
  'arrTime': '12:48:00',
  'arrDate': '2025-01-16',
  'minimumChangeDuration': 'PT9M'},
 {'name': 'Kungsbacka station',
  'id': 'A=1@O=Kungsbacka station@X=12079614@Y=57490556@U=1@L=740000161@',
  'extId': '740000161',
  'routeIdx': 2,
  'lon': 12.079614,
  'lat': 57.490556,
  'depTime': '12:59:00',
  'depDate': '2025-01-16',
  'arrTime': '12:59:00',
  'arrDate': '2025-01-16',
  'minimumChangeDur

In [599]:
[stop.get("name")for stop in stops]

['Göteborg Centralstation',
 'Mölndal station',
 'Kungsbacka station',
 'Varberg station',
 'Falkenberg station',
 'Halmstad Centralstation',
 'Laholm station',
 'Båstad station',
 'Ängelholm station',
 'Helsingborg Centralstation',
 'Landskrona station',
 'Lund Centralstation']

### B)

In [600]:
sthlm= get_lookup("Stockholm")
sthlm["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740098000'

In [601]:
trips = route_planner(740098001, 740098000)
trips

{'Trip': [{'Origin': {'name': 'Göteborg Centralstation',
    'type': 'ST',
    'id': 'A=1@O=Göteborg Centralstation@X=11973479@Y=57708895@U=1@L=740000002@',
    'extId': '740000002',
    'lon': 11.973479,
    'lat': 57.708895,
    'routeIdx': 0,
    'prognosisType': 'PROGNOSED',
    'time': '12:18:00',
    'date': '2025-01-16',
    'minimumChangeDuration': 'PT15M'},
   'Destination': {'name': 'Stockholm Centralstation',
    'type': 'ST',
    'id': 'A=1@O=Stockholm Centralstation@X=18058151@Y=59330136@U=1@L=740000001@',
    'extId': '740000001',
    'lon': 18.058151,
    'lat': 59.330136,
    'routeIdx': 4,
    'prognosisType': 'PROGNOSED',
    'time': '15:41:00',
    'date': '2025-01-16',
    'minimumChangeDuration': 'PT20M'},
   'ServiceDays': [{'planningPeriodBegin': '2025-01-02',
     'planningPeriodEnd': '2025-06-15',
     'sDaysR': 'inte varje dag',
     'sDaysI': '16. jan t o m 9. jun 2025; 22. mar t o m 20. apr 2025 mån - fre; 3. maj t o m 7. jun 2025 mån - fre; utom 18., 21. ap

In [602]:
trip= trips["Trip"][0]

In [603]:
stops= trip["LegList"]["Leg"][0]["Stops"]["Stop"]

In [604]:
[{stop["name"]: stop.get("arrTime")}for stop in stops]

[{'Göteborg Centralstation': None},
 {'Herrljunga station': '13:01:00'},
 {'Skövde Centralstation': '13:27:00'},
 {'Katrineholm Centralstation': '14:47:00'},
 {'Stockholm Centralstation': '15:41:00'}]

In [605]:
coords= []
for stop in stops:
    city= get_lookup(stop.get("name"))
    city= city["stopLocationOrCoordLocation"][0]["StopLocation"]
    coords.append((city["lon"], city["lat"]))

In [606]:
coords

[(11.973479, 57.708895),
 (13.021279, 58.079188),
 (13.853195, 58.390898),
 (16.208324, 58.996591),
 (18.058151, 59.330136)]

In [607]:
data= pd.DataFrame(coords, columns=["lon", "lat"])
data

,lon,lat
0,11.973479,57.708895
1,13.021279,58.079188
2,13.853195,58.390898
3,16.208324,58.996591
4,18.058151,59.330136


In [608]:
import folium
map= folium.Map(location= [data["lat"].mean(), data["lon"].mean()])

for lat, lon in zip(data["lat"], data["lon"]):
    folium.Marker(location= [lat, lon]).add_to(map)

map

In [609]:
map= folium.Map(location= [data["lat"].mean(), data["lon"].mean()])
coords = [[lat, lon] for lat, lon in zip(data["lat"], data["lon"])]
folium.PolyLine(locations= coords).add_to(map)

map

In [610]:
fig= px.line_mapbox(data, "lat", "lon", mapbox_style="open-street-map", zoom= 6.5, width= 800)
fig.show()

# 2


### A)

In [611]:
from datetime import datetime

In [612]:
beryll = get_lookup("Beryllgatan")

In [613]:
beryll.keys()

dict_keys(['stopLocationOrCoordLocation', 'TechnicalMessages', 'serverVersion', 'dialectVersion', 'requestId'])

In [614]:
beryll["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740059121'

In [615]:
beryll_departures= get_departures(API_KEY, 740059121)["Departure"]

In [616]:
kungs= get_lookup("Kungsportsplatsen")

In [617]:
kungs["stopLocationOrCoordLocation"][0]["StopLocation"]["extId"]

'740016358'

In [618]:
kungs_departures= get_departures(API_KEY, "740016358")["Departure"]

In [619]:
def get_departure_station(departures):
    time= int(datetime.now().strftime("%H:%M:%S").split(":")[1])
    
    for trip in departures[:5]:
        departure_time= trip.get('time').split(":")
        departure_minutes= int(departure_time[1]) - time
        print(f"{''.join(trip.get('name').split('- ')[1:])} mot {trip.get('direction')}: {departure_minutes}")

In [620]:
get_departure_station(beryll_departures)

Buss 50 mot Göteborg Brunnsparken: 3
Buss 97 mot Fiskebäcks Småbåtshamn (Göteborg kn): 4
Buss 50 mot Frölunda torg (Göteborg kn): 5
Buss 97 mot Frölunda torg (Göteborg kn): 6
Buss 50 mot Göteborg Brunnsparken: 13


In [621]:
get_departure_station(kungs_departures)

Buss 18 mot Bäckebol Norra (Göteborg kn): 0
Spårväg 10 mot Göteborg Eketrägatan: 0
Spårväg 5 mot Göteborg Östra sjukhuset: 0
Spårväg 4 mot Angered centrum (Göteborg kn): 1
Spårväg 3 mot Virginsgatan (Göteborg kn): 1
